In [67]:
import pickle
import numpy as np
import sklearn
from scipy.optimize import minimize
import scipy.sparse as sp
import sys
sys.path.append ('../modules')
import hputils

### Generate cascades

In [72]:
def simulate_multivariate_cascades (mu, 
                                    alpha, 
                                    end_time, 
                                    bandwidth=1.0, 
                                    n_realizations=50, 
                                    num_events=None, 
                                    check_stability=False, 
                                    seed=None):
    """ Modified implementation from https://github.com/stmorse/hawkes

    Parameters:
    ===========
    mu(numpy.ndarray):  the base intensity vector
    alpha (numpy.ndarray): the excitation matrix
    end_time (float) : All the generated timestamps should lie within the end_time
    bandwidth (float): the bandwidth parameter (default: 1.0)
    n_realizations (int) : the number of different cascades to be generated (default: 50)
    num_events (int) : The number of events to be generated (default: None)
                 If None, events are generated upto time `end_time`.
                 If not None, attempt to generate upto `num_events` events
                 under the condition that the time does not exceeed `end_time`.
    check_stability (bool): Before simulating, check the stability of HP by spectral analysis (default: False)

    seed (int): The seed for the random number generation (default: None)
          This ensures repeatability in the generation process.
          For debugging one should specify a constant seed.
          But otherwise the seed should be `None`.

    Returns:
    ========
    cascades (list of lists): Nested list. The inner lists are individual cascades
                        Every element of the list is a pair (source, timestamp)
                        of the generated event.
    """

    if check_stability:
        w,v = np.linalg.eig (alpha)
        max_eig = np.amax (np.abs(w))
        if max_eig >= 1:
            print (f"(WARNING) Unstable ... max eigen value is: {max_eig}")

    prng = sklearn.utils.check_random_state (seed)
    dims = mu.shape[0]
    
    if num_events is None:
        n_expected = np.iinfo (np.int32).max
    else:
        n_expected = num_events

    cascades = list ()
    for i in range (n_realizations):
        # Initialization
        cascade = list ()
        n_total = 0
        istar = np.sum(mu)
        s = hputils.draw_exponential_random_variable (1./istar, prng)
        if s <=end_time and n_total < n_expected:
            # attribute (weighted random sample, since sum(mu)==Istar)
            n0 = int(prng.choice(np.arange(dims), 1, p=(mu / istar)))
            cascade.append((n0, s))
            n_total += 1

        # value of \lambda(t_k) where k is most recent event
        # starts with just the base rate
        last_rates = mu.copy()
        dec_istar = False
        while n_total < n_expected:
            uj, tj = int (cascade[-1][0]), cascade[-1][1]
            if dec_istar:
                # if last event was rejected, decrease istar
                istar = np.sum(rates)
                dec_istar = False
            else:
                # otherwise, we just had an event, so recalc Istar (inclusive of last event)
                istar = np.sum(last_rates) + alpha[uj,:].sum()

            s += hputils.draw_exponential_random_variable (1./istar, prng)
            if s > end_time:
                break

            # calc rates at time s (use trick to take advantage of rates at last event)
            rates = mu + hputils.exp_kernel (s,tj,bandwidth) * (alpha[uj,:] + last_rates - mu)

            # attribution/rejection test
            # handle attribution and thinning in one step as weighted random sample
            diff = istar - np.sum(rates)
            n0 = int (prng.choice(np.arange(dims+1), 1, p=(np.append(rates, diff) / istar)))

            if n0 < dims:
                cascade.append((n0, s))
                # update lastrates
                last_rates = rates.copy()
                n_total += 1
            else:
                dec_istar = True

        cascades.append (cascade)
    return cascades

Let's generate a few cascades for a small (n=15) size network.

In [73]:
dims = 15
num_cascades = 100
end_time = 30
baseline = np.random.uniform (low=0.0, high=1.0/dims, size=(dims,))
adjacency = np.random.uniform (low=0.0, high=1.0/dims, size=(dims,dims))
params = (baseline, adjacency)


cascades = simulate_multivariate_cascades (baseline, adjacency, end_time, seed=1039, n_realizations=num_cascades)

### Discrete Vanilla HP

TODO: I can't figure out how to make these objects stateful (is it needed) since the optimize 
      function expects a certain input format.

In [90]:
class DVHP:
    """ Discrete vanilla hawkes process. """
    @staticmethod
    def log_likelihood_single_cascade (params,
                                       cascade, 
                                       bandwidth=1.0, 
                                       dims=5, 
                                       sign=-1.0, 
                                       epsilon=1e-5, 
                                       verbose=False):
        
        """ Calculate the log-likelihood for a single cascade
        
        Parameters:
        ===========
        params (numpy.ndarray): The parameters of the model (mu and alpha) flattened 
                                as a single array.
        cascade (dict): A sequence of events. As the events happen at discrete times, 
                        the discrete times are the keys in the dictionary. 
                        The values for each key (discrete time) in the dictionary is a 
                        list of sources who had an event at that time.
        
        bandwidth (float): The bandwidth of the exponential decay kernel (default=1.0)
        dims (int): The number of sources (default=5)
        sign (float): 1.0 for calculating positive log likelihood; -1 for calculating
                      negative log likelihood.
        epsilon (float): A small value so that we don't run into divide by zero 
                         errors (default=1e-5).
        verbose (bool): Flag to control the verbosity (default=False)
        """
        # unflatten parameters
        mu = params[0:dims]
        alpha = params[dims:].reshape (dims, dims)
        
        # initialization
        eta = np.zeros_like (mu)
        intensities = np.zeros_like (mu)
        total_intensities = np.zeros_like (mu)
        
        last_timestamp = -1
        log_intensities = 0
        for timestamp in sorted(cascade.keys()):
            if last_timestamp < 0: # means we're looking at the first set of discrete events
                eta = 0
            else:
                last_sources = cascade[last_timestamp]
                eta = hputils.exp_kernel (timestamp, last_timestamp, bandwidth) * (eta + alpha[last_sources, :].sum(axis=0))
            intensities = mu + eta
            current_sources = cascade[timestamp]
            log_intensities += np.log (intensities[current_sources] + epsilon).sum()
            total_intensities += intensities
            last_timestamp = timestamp
        
        ll = (sign) * (log_intensities - total_intensities.sum())
        return ll
    
    @staticmethod
    def grad_mu (params, 
                 cascade, 
                 bandwidth=1.0, 
                 dims=5, 
                 sign=-1.0, 
                 epsilon=1e-5, 
                 verbose=False):
        pass
    
    @staticmethod
    def grad_alpha (params, 
                    cascade, 
                    bandwidth=1.0, 
                    dims=5, 
                    sign=-1.0, 
                    epsilon=1e-5, 
                    verbose=False):
        pass
    
    @staticmethod
    def log_likelihood_many_cascades (params, 
                                      cascades, 
                                      bandwidth=1.0, 
                                      dims=5, 
                                      sign=-1.0, 
                                      epsilon=1e-5, 
                                      verbose=False):
        
        """ Calculate the log-likelihood for a multiple cascades
        
        Parameters:
        ===========
        params (numpy.ndarray): The parameters of the model (mu and alpha) flattened 
                                as a single array.
        cascades (list): A collection of cascades. Each cascade (item in the list) 
                         is a dictionary.
        bandwidth (float): The bandwidth of the exponential decay kernel (default=1.0)
        dims (int): The number of sources (default=5)
        sign (float): 1.0 for calculating positive log likelihood; -1 for calculating
                      negative log likelihood.
        epsilon (float): A small value so that we don't run into divide by zero 
                         errors (default=1e-5).
        verbose (bool): Flag to control the verbosity (default=False)
        """
        
        n_cascades = len (cascades)
        total_log_likelihood = 0.0
        for i in range (n_cascades):
            log_likelihood = DVHP.log_likelihood_single_cascade (params, \
                                                                 cascades[i], \
                                                                 bandwidth, \
                                                                 dims, \
                                                                 sign, \
                                                                 verbose)
            total_log_likelihood += log_likelihood

        if verbose: print (total_log_likelihood/n_cascades)
        return total_log_likelihood/n_cascades
    
    @staticmethod
    def estimate (cascades, 
                  log_likelihood, 
                  gradient=None, 
                  bandwidth=1.0, 
                  dims=5):
        bounds = [(0,None) for i in range (dims + (dims**2))] # set the non-negativity bounds on the parameters
        params = np.random.uniform (0, 1, size=dims + dims ** 2) # random initialization of the parameters
        sign = -1.0 # Multiply so that we can minimize negative log likelihood
        epsilon = 1e-5
        result = minimize (log_likelihood,
                           params,
                           args=(cascades, bandwidth, dims, sign, epsilon, False),
                           method='L-BFGS-B',
                           jac=gradient,
                           bounds=bounds,
                           options={'ftol': 1e-10, "maxls": 50, "maxcor":50, "maxiter":500, "maxfun": 500, "disp": True})
        mu = result.x[:dims]
        alpha = result.x[dims:].reshape (dims,dims)
        return mu, alpha

### Continuous Vanilla HP

In [77]:
def get_end_time (cascade, epsilon=0.001):
    end_time = max ([t for _, t in cascade])
    return end_time + epsilon

In [96]:
class CVHP:
    """ Continuous timestamps vanilla Hawkes process"""
    @staticmethod
    def precompute_kernel_sums_single_cascade (cascade, 
                                               dims,
                                               bandwidth):
        """ This function calculates the intermediate quantity which is the
            unweighted cumulative intensity of past events.
            R[i][j,t] is defined as the excitation at any given time.
            i is the node that recieves the excitation
            j is the node that has had past events
            t is the time at which we calculate the excitation
            R[i][j,t] = K(t-lasttime(i)) * R[i][j,lasttime(i)] ... i=j
                      = K(t-lasttime(i)) * R[i][j,lasttime(i)] + \sum_{e:lasttime(i) < t_e < t} K(t-t_e) ... i!=j

        Parameters:
        ===========
        
        cascade (list): A sequence of events, i.e., source, timestamp pairs.
        dims (int): The number of sources (or nodes)
        bandwidth (float): The bandwidth parameter for the exponential kernel function.
        
        Notes:
            This function assumes the exponential decay kernel which simplifies the calculation.
            Other kernels may not work out in this way.
            
        """
        # initialization
        R = {i: sp.lil_matrix((dims, len (cascade) + 1)) for i in range (dims)}
        lastseen = {i: (-np.inf, 0) for i in range (dims)}

        for i, event in enumerate (cascade):
            src, curr = int (event[0]), event[1]
            R[src][:,i+1] = hputils.exp_kernel (curr, lastseen[src][0], bandwidth) * R[src][:, lastseen[src][1] + 1]

            # do per node differently
            subseq = cascade[lastseen[src][1]:i]
            for e in subseq:
                n, prev = int (e[0]), e[1]
                R[src][n, i+1] += hputils.exp_kernel (curr, prev, bandwidth)

            # update the last seen event for the node
            lastseen[src] = (curr, i)

        return R

    @staticmethod
    def precompute_kernel_sums_multiple_cascades (cascades,
                                                  dims=5,
                                                  bandwidth=1.0):
        return [CVHP.precompute_kernel_sums_single_cascade (cascade, dims, bandwidth) for cascade in cascades]

    @staticmethod
    def log_likelihood_single_cascade (params,
                                       cascade,
                                       kernel_sum,
                                       bandwidth,
                                       dims,
                                       end_time,
                                       sign,
                                       verbose):
        """ Computes the multivariate likelihood objective for a single cascade.

        Parameters:
        ===========
        params (numpy.ndarray): The parameters of the objective function flattened into
                                a 1D array.
        cascade (list): Each cascade is a list of events, i.e., (source, timestamp)
                        pairs.

        kernel_sum (dict): Refer to Section 5.3 in Goel et. al. (2016)
                           key is the node index and value is sparse.lil_matrix
                           of dimension nodes * (events + 1)

        bandwidth (float): The bandwidth of the exponential decay kernel.
        dims (int): The number of unique sources (or nodes).
        end_time (float): The end time of the window such that all the event timestamps
                          are within this end time.
        sign (float): The sign that needs to be multiplied to the objective function.
                      The value can be either 1.0 (if calculating log likelihood) or
                      -1.0 (if calculating the negative log likelihood).
        verbose (bool): If True, print diagnostic messages; otherwise print nothing.
        """
        # some offset so we don't evaluate np.log (0) at any stage
        epsilon = 1e-5

        # unflatten parameters
        mu = params[0:dims]
        alpha = params[dims:].reshape (dims, dims)

        # initialize
        term1 = term3 = 0.0

        # survival that depends on the base intensity. since base intensity is
        # constant, this part can be pulled out of the for loop
        term2 = np.sum (mu) * end_time
        for i, event in enumerate (cascade):
            src, curr = int (event[0]), event[1]
            term1 += np.log (mu[src] + sum([alpha[j,src] * kernel_sum[src][j,i+1] for j in range (dims)]) + epsilon)
            term3 += (alpha[src,:] * (1 - hputils.exp_kernel (end_time, curr, bandwidth))).sum()
        ll = (sign) * (term1 - term2 - term3)
        if verbose: print (ll, term1, term2, term3)
        return ll

    @staticmethod
    def log_likelihood_many_cascades (params, 
                                      cascades, 
                                      kernel_sums, 
                                      bandwidth=1.0, 
                                      dims=5, 
                                      sign=-1.0, 
                                      epsilon=1e-5, 
                                      verbose=False):
        """ Computes the multivariate log likelihood objective for a set of cascades

        Parameters:
        ===========

        params (numpy.ndarray): The parameters of the objective function flattened into
        a 1D array.

        cascades (list of lists): A nested list representation of cascades. Each
                          cascade is a list of events, i.e., (source, timestamp) pairs.

        kernel_sums (list of dicts): Refer to Section 5.3 in Goel et. al. (2016); one per cascade.
        bandwidth (float): The bandwidth of the exponential decay kernel.
        dims (int): The number of unique sources (or nodes).
        sign (float): The sign that needs to be multiplied to the objective function.
                      The value can be either 1.0 (if calculating log likelihood) or
                      -1.0 (if calculating the negative log likelihood).

        verbose (bool): If True, print diagnostic messages; otherwise print nothing.

        """
            
        n_cascades = len (cascades)
        total_log_likelihood = 0.0
        for i in range (n_cascades):
            end_time = get_end_time (cascades[i])
            log_likelihood = CVHP.log_likelihood_single_cascade (params, \
                                                                 cascades[i], \
                                                                 kernel_sums[i], \
                                                                 bandwidth, \
                                                                 dims, \
                                                                 end_time, \
                                                                 sign, \
                                                                 verbose)
            
            total_log_likelihood += log_likelihood
            
        if verbose: print (total_log_likelihood/n_cascades)
        return total_log_likelihood/n_cascades
    
    @staticmethod
    def estimate (cascades, log_likelihood, gradient=None, dims=5, bandwidth=1.0):
        bounds = [(0,None) for i in range (dims + (dims**2))] # set the non-negativity bounds on the parameters
        params = np.random.uniform (0, 1, size=dims + dims ** 2) # random initialization of the parameters
        sign = -1.0 # Multiply so that we can minimize negative log likelihood
        epsilon = 1e-5
        kernel_sums = CVHP.precompute_kernel_sums_multiple_cascades (cascades, dims=dims, bandwidth=bandwidth)
        result = minimize (log_likelihood,
                           params,
                           args=(cascades, kernel_sums, bandwidth, dims, sign, epsilon, False),
                           method='L-BFGS-B',
                           jac=gradient,
                           bounds=bounds,
                           options={'ftol': 1e-10, "maxls": 50, "maxcor":50, "maxiter":500, "maxfun": 500, "disp": True})
        mu = result.x[:dims]
        alpha = result.x[dims:].reshape (dims,dims)
        return mu, alpha

In [81]:
def to_discrete (old_cascade):
    cascade = dict ()
    for src, timestamp in [(src, int (timestamp)) for src, timestamp in old_cascade]:
        if timestamp not in cascade:
            cascade[timestamp] = list ()
        cascade[timestamp].append (src)
    return cascade

In [91]:
discrete_cascades = [to_discrete (cascade) for cascade in cascades]
mu, alpha = DVHP.estimate (discrete_cascades, DVHP.log_likelihood_many_cascades, dims=dims)
print(np.linalg.norm (baseline - mu))
print(np.linalg.norm (adjacency - alpha))

0.8660343166379001
3.925330459201396


In [94]:
mu, alpha = CVHP.estimate (cascades, CVHP.log_likelihood_many_cascades, dims=dims)
print(np.linalg.norm (baseline - mu))
print(np.linalg.norm (adjacency - alpha))

0.6275673486323886
3.2028366908809938


In [102]:
print (DVHP.log_likelihood_many_cascades (np.concatenate ((baseline, np.ravel (adjacency))), discrete_cascades, dims=dims))
print (CVHP.log_likelihood_many_cascades (np.concatenate ((baseline, np.ravel (adjacency))), cascades, 
                                          CVHP.precompute_kernel_sums_multiple_cascades(cascades, dims=dims), dims=dims))

103.71235251824885
109.42192956567641


As a sanity check, let's make the discretization more granular. Then the log-likelihood should converge.

TODO

## Discrete Coarse HP

In [114]:
class DCHP:
    """ Discrete coarse hawkes process. """
    @staticmethod
    def log_likelihood_single_cascade (params, 
                                       cascade, 
                                       bandwidth=1.0, 
                                       dims=5, 
                                       sign=-1.0, 
                                       epsilon=1e-5, 
                                       verbose=False):
        # unflatten parameters
        mu = params[0*dims:1*dims]
        b = params[1*dims:2*dims]
        c = params[2*dims:3*dims]
        s = params[3*dims:4*dims]
        
        # initialization
        eta = np.zeros_like (mu)
        intensities = np.zeros_like (mu)
        total_intensities = np.zeros_like (mu)
        
        last_timestamp = -1
        log_intensities = 0 
        se_gate = np.eye (dims)
        
        for timestamp in sorted(cascade.keys()):
            if last_timestamp < 0:
                eta = 0
            else:
                last_sources = cascade[last_timestamp]
                aggregator = np.zeros_like (mu)
                for last_source in last_sources:
                    aggregator += ((b[last_source] * c) + se_gate[last_source,:])
                    
                eta = hputils.exp_kernel (timestamp, last_timestamp, bandwidth) * (eta + aggregator)
            intensities = mu + eta
            current_sources = cascade[timestamp]
            log_intensities += np.log (intensities[current_sources] + epsilon).sum()
            total_intensities += intensities
            last_timestamp = timestamp
        
        ll = (sign) * (log_intensities - total_intensities.sum())
        return ll

    @staticmethod
    def grad_mu (params, cascade, bandwidth=1.0, dims=5, sign=-1.0, epsilon=1e-5, verbose=False):
        pass
    
    @staticmethod
    def grad_alpha (params, cascade, bandwidth=1.0, dims=5, sign=-1.0, epsilon=1e-5, verbose=False):
        pass
    
    @staticmethod
    def log_likelihood_many_cascades (params, 
                                      cascades, 
                                      bandwidth=1.0, 
                                      dims=5, 
                                      sign=-1.0, 
                                      epsilon=1e-5, 
                                      verbose=False):
        n_cascades = len (cascades)
        total_log_likelihood = 0.0
        for i in range (n_cascades):
            log_likelihood = DCHP.log_likelihood_single_cascade (params, \
                                                                 cascades[i], \
                                                                 bandwidth, \
                                                                 dims, \
                                                                 sign, \
                                                                 epsilon, \
                                                                 verbose)
            total_log_likelihood += log_likelihood

        if verbose: print (total_log_likelihood/n_cascades)
        return total_log_likelihood/n_cascades
    
    @staticmethod
    def estimate (cascades, 
                  log_likelihood, 
                  gradient=None, 
                  bandwidth=1.0, 
                  dims=5):
        bounds = [(0,None) for i in range (4*dims)] # set the non-negativity bounds on the parameters
        params = np.random.uniform (0, 1, size=4*dims) # random initialization of the parameters
        sign = -1.0 # Multiply so that we can minimize negative log likelihood
        epsilon = 1e-5
        result = minimize (log_likelihood,
                           params,
                           args=(cascades, bandwidth, dims, sign, epsilon, False),
                           method='L-BFGS-B',
                           jac=gradient,
                           bounds=bounds,
                           options={'ftol': 1e-10, "maxls": 50, "maxcor":50, "maxiter":500, "maxfun": 500, "disp": True})
        mu = result.x[:dims]
        b = result.x[dims:2*dims]
        c = result.x[2*dims:3*dims]
        s = result.x[3*dims:4*dims]
        return mu,b,c,s

## Continuous Coarse HP

In [119]:
class CCHP:
    @staticmethod
    def log_likelihood_single_cascade (params,
                                       cascade,
                                       kernel_sum,
                                       bandwidth,
                                       dims,
                                       end_time,
                                       sign,
                                       verbose):

        """ Computes the multivariate likelihood objective for a single cascade.

        Parameters:
        ===========
        params (numpy.ndarray): The parameters of the objective function flattened into
                                a 1D array.
        cascade (list): Each cascade is a list of events, i.e., (source, timestamp)
                        pairs.

        kernel_sum (dict): Refer to Section 5.3 in Goel et. al. (2016)
                           key is the node index and value is sparse.lil_matrix
                           of dimension nodes * (events + 1)

        bandwidth (float): The bandwidth of the exponential decay kernel.
        dims (int): The number of unique sources (or nodes).
        end_time (float): The end time of the window such that all the event timestamps
                          are within this end time.
        sign (float): The sign that needs to be multiplied to the objective function.
                      The value can be either 1.0 (if calculating log likelihood) or
                      -1.0 (if calculating the negative log likelihood).
        verbose (bool): If True, print diagnostic messages; otherwise print nothing.
        """

        # some offset so we don't evaluate np.log (0) at any stage
        epsilon = 1e-5

        # unflatten parameters
        mu = params[0*dims:1*dims]
        b = params[1*dims:2*dims]
        c = params[2*dims:3*dims]
        s = params[3*dims:4*dims]

        # initialize
        term1 = term3 = 0.0

        # survival that depends on the base intensity. since base intensity is
        # constant, this part can be pulled out of the for loop
        term2 = np.sum (mu) * end_time
        for i, event in enumerate (cascade):
            src, curr = int (event[0]), event[1]
            term1 += np.log (mu[src] + sum([b[j]*c[src] * kernel_sum[src][j,i+1] if not j == src else (b[j]*c[src] + s[j]) * kernel_sum[j][j,i+1] for j in range (dims)]) + epsilon)
            v = b[src] * c
            v[src] += s[src]
            term3 += (v * (1 - hputils.exp_kernel (end_time, curr, bandwidth))).sum()

        ll = (sign) * (term1 - term2 - term3)
        if verbose: print (ll, term1, term2, term3)
        return ll

    @staticmethod
    def log_likelihood_many_cascades (params, 
                                      cascades, 
                                      kernel_sums, 
                                      bandwidth=1.0, 
                                      dims=5, 
                                      sign=-1.0, 
                                      epsilon=1e-5, 
                                      verbose=False):
        """ Computes the multivariate log likelihood objective for a set of cascades

        Parameters:
        ===========

        params (numpy.ndarray): The parameters of the objective function flattened into
        a 1D array.

        cascades (list of lists): A nested list representation of cascades. Each
                          cascade is a list of events, i.e., (source, timestamp) pairs.

        kernel_sums (list of dicts): Refer to Section 5.3 in Goel et. al. (2016); one per cascade.
        bandwidth (float): The bandwidth of the exponential decay kernel.
        dims (int): The number of unique sources (or nodes).
        sign (float): The sign that needs to be multiplied to the objective function.
                      The value can be either 1.0 (if calculating log likelihood) or
                      -1.0 (if calculating the negative log likelihood).

        verbose (bool): If True, print diagnostic messages; otherwise print nothing.

        """
            
        n_cascades = len (cascades)
        total_log_likelihood = 0.0
        for i in range (n_cascades):
            end_time = get_end_time (cascades[i])
            log_likelihood = CCHP.log_likelihood_single_cascade (params, \
                                                                 cascades[i], \
                                                                 kernel_sums[i], \
                                                                 bandwidth, \
                                                                 dims, \
                                                                 end_time, \
                                                                 sign, \
                                                                 verbose)

            total_log_likelihood += log_likelihood

        if verbose: print (total_log_likelihood/n_cascades)
        return total_log_likelihood/n_cascades
    
    @staticmethod
    def estimate (cascades, 
                  log_likelihood, 
                  gradient=None, 
                  bandwidth=1.0, 
                  dims=5):
        bounds = [(0,None) for i in range (4*dims)] # set the non-negativity bounds on the parameters
        params = np.random.uniform (0, 1, size=4*dims) # random initialization of the parameters
        sign = -1.0 # Multiply so that we can minimize negative log likelihood
        epsilon = 1e-5
        kernel_sums = CVHP.precompute_kernel_sums_multiple_cascades (cascades, dims=dims, bandwidth=bandwidth)
        result = minimize (log_likelihood,
                           params,
                           args=(cascades, kernel_sums, bandwidth, dims, sign, epsilon, False),
                           method='L-BFGS-B',
                           jac=gradient,
                           bounds=bounds,
                           options={'ftol': 1e-10, "maxls": 50, "maxcor":50, "maxiter":500, "maxfun": 500, "disp": True})
        mu = result.x[:dims]
        b = result.x[dims:2*dims]
        c = result.x[2*dims:3*dims]
        s = result.x[3*dims:4*dims]
        return mu,b,c,s

In [108]:
dims = 15
num_cascades = 100
end_time = 30
baseline = np.random.uniform (low=0.0, high=1.0/dims, size=(dims,))
trans = np.random.uniform (low=0.0, high=2.0/dims, size=(dims,))
recep = np.random.uniform (low=0.0, high=2.0/dims, size=(dims,))
selfe = np.random.uniform (low=0.0, high=1.0/dims, size=(dims,))
adjacency = np.outer (trans, recep) + np.diag (selfe)
#adjacency = np.random.uniform (low=0.0, high=1.0/dims, size=(dims,dims))
params = (baseline, adjacency)


cascades = simulate_multivariate_cascades (baseline, adjacency, end_time, seed=1039, n_realizations=num_cascades)

In [112]:
discrete_cascades = [to_discrete (cascade) for cascade in cascades]
mu, b,c,s = DCHP.estimate (discrete_cascades, DCHP.log_likelihood_many_cascades, dims=dims)
print(np.linalg.norm (baseline - mu))
print(np.linalg.norm (trans - b))
print(np.linalg.norm (recep - c))
print(np.linalg.norm (selfe - s))

0.1838473459241123
0.255776736016798
0.22968400886594822
2.091889779392128


In [120]:
mu, b,c,s = CCHP.estimate (cascades, CCHP.log_likelihood_many_cascades, dims=dims)
print(np.linalg.norm (baseline - mu))
print(np.linalg.norm (trans - b))
print(np.linalg.norm (recep - c))
print(np.linalg.norm (selfe - s))

0.7662275456080186
1.5706195312993447
1.60361401705122
2.0505942432004733
